# Training and saving the CNN model
* Dataset: CIFAR10
* Model: MobileNetV2 + hidden-layer + full-coupled layer
* Framework: Tensorflow

# Setup the hyperparameters

In [ ]:
img_size = 32
n_channels = 3
batch_size = 32
n_classes = 10
epochs = 10
n_hidden = 256
init_lr = 0.001

## Preparing

In [10]:
import numpy as np
import tensorflow as tf

# load cifar10 dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

# normalize the data
x_train = x_train / 255.0
x_test = x_test / 255.0

# one-hot encode the labels
y_train = tf.keras.utils.to_categorical(y_train, n_classes)
y_test = tf.keras.utils.to_categorical(y_test, n_classes)

## Build the model

In [7]:
# data augmentation
datagen = tf.keras.layers.sequential([
    tf.keras.layers.RandomRotation(20),
    tf.keras.layers.RandomWidthShift(0.2),
    tf.keras.layers.RandomHeightShift(0.2),
    tf.keras.layers.RandomShear(10),
    tf.keras.layers.RandomZoom(0.2),
    tf.keras.layers.RandomFlip('horizontal')
])

# rescale pixel values to [-1, 1] for using mobilnetV2
preprosess_input = tf.keras.applications.mobilenet_v2.preprocess_input

# create the base model
base_model = tf.keras.applications.MobileNetV2(input_shape=(img_size, img_size, n_channels),
                                               include_top=False,
                                               weights='imagenet')
base_model.trainable = False # freeze the base model

# 5x5 dimension convolutional layer convert the image to 1280 vector
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()

# build the model
inputs = tf.keras.Input(shape=(img_size, img_size, n_channels))
x = datagen(inputs)
x = preprosess_input(x)
x = base_model(x, training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dense(n_hidden, activation='relu')(x)
x = tf.keras.layers.Dropout(0.25)(x)
outputs = tf.keras.layers.Dense(n_classes, activation='softmax')(x)
model = tf.keras.Model(inputs, outputs)

# set up the learning rate scheduler
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    init_lr,
    decay_steps=10000,
    decay_rate=0.99)

# set up the optimizer
opt = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

# set up the loss
loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

# compile the model
model.compile(optimizer=opt,
              loss=loss,
              metrics=['accuracy'])
model.summary()

## Train the model

In [ ]:
# set up the early stopping callback
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

# train the model
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_data=(x_test, y_test),
                    callbacks=[early_stopping])

## Evaluate the model

In [ ]:
import matplotlib.pyplot as plt

# set up the parameters
loss = history.history['loss']
val_loss = history.history['val_loss']
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

# Evaluate the model on the test data using `evaluate`
loss, acc = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', loss.round(4))
print('Test accuracy:', acc.round(4))

## Save the model

In [ ]:
from google.colab import files

# save the model
model.save('cifar10_model.h5', include_optimizer=False)
files.download('cifar10_model.h5')